In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('manhattan_rents - Data.csv')

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.reset_index(drop=True, inplace=False)

In [ ]:
df.info()

In [ ]:
df.drop(df.index[df['bedrooms'] % 1 != 0], inplace = True)

In [ ]:
df.describe()

In [ ]:
def scaleRent (df):
    for row in range(len(df.index)):
        if df.iat[row, 2] == 0:
            df.iat[row, 1] = df.iat[row, 1]/1
        else:
            df.iat[row, 1] = df.iat[row, 1]/df.iat[row, 2]
    return df

In [ ]:
df = scaleRent (df)

In [ ]:
def normalizingData (df):
    for column in df:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())  
    return df

In [ ]:
df = normalizingData (df)

In [ ]:
df.head()

In [ ]:
cor = df.corr()

In [ ]:
sns.heatmap(cor, annot = True)

In [ ]:
df.info()

In [ ]:
def testTrainSplit (df):
    df = df.sample(frac = 1, ignore_index=True)
    length = len(df.index)
    splitPoint = round(length * 0.70)
    training = df.iloc[:splitPoint, :]
    testing = df.iloc[splitPoint:, :]
    return training, testing

In [ ]:
training, testing = testTrainSplit (df)

In [ ]:
training.info()

In [ ]:
testing.info()

In [ ]:
def fittingPlane (df, wBList):
    listOfFits = []
    W3 = -1
    for wBPair in wBList:
        W1 = wBPair["W1"]
        W2 = wBPair["W2"]
        B = wBPair["B"]
        dSum = 0
        for index, row in df.iterrows():
            X1 = row["bedrooms"]
            X2 = row["size_sqft"]
            Y = row["rent"]
            
            distance = (abs((W1 * X1) + (W2 * X2) + (W3 * Y) + B)) / (((W1**2) + (W2**2) + (W3**2))**0.5)
            
            dSum = dSum + distance
        template = {"Distance Sum": dSum, "W1": W1, "W2": W2, "B": B}
        listOfFits.append(template)
    dictionary = findSmallestSum (listOfFits)
    
    return dictionary

In [ ]:
def findSmallestSum (listOfFits):
    lowest = float ('inf')
    for sum in listOfFits:
        if sum['Distance Sum'] < lowest:
            lowest = sum['Distance Sum']
            dictionary = sum
    return dictionary

In [ ]:
def creatingWeights (lWeight, hWeight, lYIntercept, hYIntercept):
    wBList = []
    for firstWeight in range(lWeight, hWeight + 1):
        for secondWeight in range (lWeight, hWeight + 1):
            for yIntercept in range (lYIntercept, hYIntercept + 1):
                template = {"W1": firstWeight/10, "W2": secondWeight/10, "B" : yIntercept/10}
                wBList.append(template)
    return wBList

In [ ]:
wBList = creatingWeights (-5, 10, -5, 10)

In [ ]:
dictionary = fittingPlane (df, wBList)

In [ ]:
dictionary